# Xóa Dữ Liệu Gold Tables

In [1]:
from pyspark.sql import SparkSession
import os

os.environ['AWS_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'admin123'

spark = (
    SparkSession.builder.appName("Clean_Gold_Tables")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "1536m")
    .config("spark.executor.cores", "2")
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://gold/")
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.sql.catalog.nessie.s3.region", "us-east-1")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")
    .config("spark.executorEnv.AWS_REGION", "us-east-1")
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", "admin")
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", "admin123")
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print("✓ Spark Session khởi tạo!")


25/12/06 12:23:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


✓ Spark Session khởi tạo!


## Xem trạng thái hiện tại

In [2]:
tables = ["Entity", "Topic", "Post", "Post_Entity", "Post_Topic"]

print("="*60)
for table_name in tables:
    try:
        count = spark.table(f"nessie.gold_result_model_multi_task.{table_name}").count()
        print(f"{table_name}: {count:,} records")
    except:
        print(f"{table_name}: Không tồn tại")
print("="*60)


Entity: 7,537 records
Topic: 10 records
Post: 0 records
Post_Entity: 0 records


[Stage 12:>                                                         (0 + 1) / 1]

Post_Topic: 0 records


## Xóa dữ liệu (đổi CONFIRM = True)

In [3]:
CONFIRM = True  # Đổi thành True để xóa

if not CONFIRM:
    print("⚠️ Đổi CONFIRM = True để xác nhận xóa")
else:
    print("="*60)
    print("BẮT ĐẦU XÓA DỮ LIỆU")
    print("="*60)
    
    for table_name in tables:
        try:
            full_name = f"nessie.gold_result_model_multi_task.{table_name}"
            before = spark.table(full_name).count()
            spark.sql(f"DELETE FROM {full_name}")
            after = spark.table(full_name).count()
            print(f"✓ {table_name}: Đã xóa {before:,} records")
        except Exception as e:
            print(f"✗ {table_name}: {e}")
    
    print("="*60)
    print("✓ HOÀN THÀNH!")
    print("="*60)


BẮT ĐẦU XÓA DỮ LIỆU
✓ Entity: Đã xóa 7,537 records


✓ Topic: Đã xóa 10 records
✓ Post: Đã xóa 0 records
✓ Post_Entity: Đã xóa 0 records
✓ Post_Topic: Đã xóa 0 records
✓ HOÀN THÀNH!


## Dừng Spark

In [4]:
spark.stop()
print("✓ Đã dừng Spark Session!")


✓ Đã dừng Spark Session!
